In [2]:
# Langchain Simple Agent

# Import required libraries

# For calling the Openai API
from langchain_openai import ChatOpenAI # LLM BRAIN - ENGINE
# # For calling the Anthropic API
# from langchain_anthropic import AnthropicChat
# # For Azure OpenAI API
# from langchain_azure_openai import AzureChatOpenAI
# # For calling the Google API
# from langchain_google import ChatGoogle
# # For calling the Ollama API
# from langchain_ollama import OllamaChat
# # For calling the Cohere API
# from langchain_cohere import CohereChat
# # For calling the HuggingFace API
# from langchain_huggingface import HuggingFaceChat
# # For calling the Google Gemini API
# from langchain_gemini import GeminiChat

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder # MESSAGES - CHASSIS
# ChatPromptTemplate: It is used to create a prompt template for chat models. This helps in structuring the input for the model.
# MessagesPlaceholder: It is used to create a placeholder for messages in the prompt template.

from langchain.tools import tool # HANDS - DASHBOARD
# tool: It is used to define a tool that can be called by the agent. This allows the agent to perform specific actions or retrieve information.

from langchain.agents import create_tool_calling_agent, AgentExecutor
# create_tool_calling_agent: It is used to create an agent that can call tools.
# AgentExecutor: It is used to execute the agent with the provided tools and model.

c:\Users\Admin\Desktop\IITM AI\llm-applications-module-4\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader # a class used to load text from PDF files

# Imports a powerful text-splitting tool that breaks large documents into manageable chunks for embedding.
# LLMs have context length limits, so splitting documents into chunks is essential.
# Why recursive?
# It tries to split at logical boundaries (like paragraphs, sentences, etc.) before falling back to raw character limits.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# OpenAIEmbeddings - to convert text into vector format using OpenAI’s embedding API
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from langchain.vectorstores import FAISS # a fast in-memory vector search engine used for semantic search.

"""Imports RetrievalQA, a chain that connects:
-A retriever (like FAISS)
-A language model (like GPT): To answer questions using retrieved context.

This is the heart of a RAG system — combining search + generation"""
from langchain.chains import RetrievalQA

In [4]:
# Step 1: Load environment variables from a .env file and set up API keys and other configurations
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
#step 2: Load and preprocess the document
pdf_path = "docs/sirca-hr.pdf"

try:
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} pages from the document.")
except Exception as e:
    print(f"Error loading document: {e}")

Loaded 24 pages from the document.


In [6]:
#step 3: Split the document into smaller chunks for processing
#RecursiveCharacter text splitter tries to split at logical boundaries (like paragraphs, sentences, etc.) before falling back to raw character limits.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Maximum size of each chunk
    chunk_overlap=100  # Overlap between chunks to maintain context
)
#chunk has text , vector representation and metadata
#chuking is important because LLMs have context length limits
chunks = text_splitter.split_documents(documents)
print(f"Split document into {len(chunks)} chunks.")
print(chunks[0])  # Print the first chunk to verify

Split document into 122 chunks.
page_content='SPIL Corporate HR Policies  
 
 
SIRCA PAINTS INDIA LTD 
NEW DELHI  
 
 
 
 
CORPORATE  
  HUMAN RESOURCES 
POLICIES & MANUALS' metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2020-08-26T06:56:00+00:00', 'author': 'hr', 'moddate': '2020-08-26T06:56:00+00:00', 'source': 'docs/sirca-hr.pdf', 'total_pages': 24, 'page': 0, 'page_label': '1'}


In [7]:
#Step 4 : Create embeddings for the chunks using OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY) #text-embedding-ada-002

print(embeddings.model)

text-embedding-ada-002


In [8]:
#step 5: Create a vector store using FAISS to store and retrieve embeddings
from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(chunks, embeddings)
print("Vector store created with FAISS.")
#save the vector store to disk
#vector_store.save_local("faiss_index")


Vector store created with FAISS.


In [9]:
#step 6: Create a RetrievalQA chain that uses the vector store to answer questions
llm = ChatOpenAI(model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
#available search types: similarity, mmr, hybrid

rag_chain = RetrievalQA.from_chain_type(
    llm=llm, # LLM model
    retriever=retriever,# Retriever to fetch relevant documents
    return_source_documents=False # Whether to return source documents along with the answer
)
print("RAG chain created.")

RAG chain created.


In [14]:
def answer_question(question: str):
    """Function to answer a question using the RAG chain."""
    result = rag_chain.invoke({"query":  question})
    answer = result['result']
    #source_docs = result['source_documents']
    
    print(f"Question: {question}\n")
    print(f"Answer: {answer}\n")
    #print("Source Documents:")
    #for i, doc in enumerate(source_docs):
     #   print(f"Document {i+1}:\n{doc.page_content}\n")
    
   # return answer, #source_docs

In [15]:
#step 7: Test the RAG system with a sample question
sample_question = "What are the key responsibilities of the HR department as mentioned in the document?"
answer_question(sample_question)


Question: What are the key responsibilities of the HR department as mentioned in the document?

Answer: The key responsibilities of the HR department mentioned in the document include:

1. Initiating and managing the recruitment process by sourcing candidates from various channels such as employment sites, internal job postings, referral schemes, campus recruitment, and placement agencies.
2. Working with department heads to identify staffing needs based on project requirements.
3. Ensuring proper documentation and statutory forms are completed by new hires on their joining date.
4. Overseeing the induction and on-boarding process for new employees.

These responsibilities focus on efficiently staffing the organization and ensuring that processes like onboarding and documentation are handled effectively to support the company's growth and adherence to internal policies.



In [16]:
#ask another question
another_question = "What does SPIL stand for?"
answer_question(another_question)

Question: What does SPIL stand for?

Answer: SPIL stands for Sirca Paints India Limited.



In [17]:
question = "what are the working hours mentioned in the document?"
answer_question(question)

Question: what are the working hours mentioned in the document?

Answer: The working hours mentioned in the document are:
- 9:30 AM to 6:00 PM
- 10:00 AM to 6:30 PM

